# Boltzmann portfolios

### Part 1: Single decision relationship to Markowitz

We develop an alternative to the traditional offline mean-variance
framework ("*Markowitz*" portfolios) called ***Boltzmann*** portfolios
which addresses growth of wealth and its uncertainties from the
standpoint of cross-entropy and optimal sequential decisions.
The improved result is a faster online algorithm which is more robust.

Boltzmann portfolios rely on geometric mean rates since they 
are optimal under logarithmic utility in the mean-variance framework. 
Accuracy on risk estimation has been improved by using our research on
Gaussian mixtures presented in https://git.io/gmix --
specifically, our function gemrat() computes the geometric mean
rate by taking into account the fourth central moment, kurtosis.
This is crucial for *leptokurtotic* ("fat-tailed") assets.

After the geometric mean rates of the underlying assets
have been estimated, we use the covariance matrix to inform us
of possible inter-correlations we can exploit to
minimize the variance of the Boltzmann portfolio.
Such minimization in fact maximizes the portfolio's terminal value
over multiple periods.

Markowitz portfolios are constructed in the arithmetic mean-variance
framework for a *single-period*. They are fragile to changing
market conditions out-of-sample, much like elegant battle strategies
which crumble under harsh war conditions.
In contrast, Boltzmann portfolios are designed to be
*adaptive* over multiple periods to maximize final wealth.
Techniques have been borrowed from Bayesian and reinforcement learning.

Multi-period process will be covered in Part 2.
Sequential decision-making using Boltzmann porfolio weights
will be covered in another notebook, Part 3.
The mathematical proofs will be given in Part 4.

*Dependencies:*

- Repository: https://github.com/rsvp/fecon235
     
*CHANGE LOG*

    2017-07-08  Add conceptual explanations.
    2017-06-28  Refactor subroutines to lib/ys_prtf_boltzmann.py
    2017-06-27  Rewrite relying more arrays than lists, for speed later.
    2017-06-26  Rough draft, test subroutines.

## Introduction


Traditional portfolio allocation (see References) develop
what is generally known as the "**efficient frontier**"
pictured here:

![Efficient mean-variance frontier](http://webpage.pace.edu/pviswanath/notes/investments/gif/assetalloc77.gif)

A student in finance may first believe the frontier's usefulness
in portfolio management, but later as a professional will perhaps
voice a **critique** along these lines:

> What I now find problematic is the theoretical concept that one
needs to only pick a point along the top of the bullet to achieve
the optimal balance between risk and reward.
Yeah, like it's so easy to just slide your finger along the curve
and select at your leisure what you'd like to earn on your money.
The focus on past prices that make up these curves
and the assumption that the covariance structure is stationary
in perpetuity is so wrong in practice.
What is rarely talked about is that the future will
NOT fall along that line. It is nice to know what you
could have earned, but what you will earn surely will not be so
easily cherry-picked off of some rear-view historic curve.
--(edited Market Tech comment on 2015-04-10)

Please pay particular attention to the apex of the hyperbolic curve 
(the "bullet") which indicates the point of **global minimum variance**.
We shall borrow that, and ignore the remaining CAPM machinery since we will
not be interested here in asset pricing relative to the market portfolio.

### Reframing the issues

One virtually has no control over how the assets perform and interact.
Only the portfolio allocation over time is in our decision set.

Let's recast the underlying assets as *agents* which supposedly will
help increase our wealth. Our task will be to select the *expert(s)*
among the agents and to allocate portions of our current wealth.

To discriminate among the agents we need their performance metrics.
Since our objective is to maximize future wealth, the
optimal metric is the geometric mean rate of each agent.
From our research we know how to include risks, including
leptokurtotic events ("fat tails"), into that single metric.

There is evidence that the performance of some agents are
inter-correlated. Therefore, rather than select a single expert,
we choose to diversify our bets among a few agents,
and call that our "portfolio."
To maximize the geometric mean rate of the portfolio,
the second order condition is to minimize its variance.
That problem is easily solved by borrowing the weights
of what is known as the "Markowitz global minimum variance portfolio."

Those weights depend on the covariance structure of the
agents' performance which is unfortunately not stable over time.
There may be some information which can be exploited
to tilt our bets favorably.

```
    prices ---> cov ---> globalw
      |                    |
      |                  trimit  <-- floor
      |                  renormalize
      |                    |
      v                    v
      |                    |
    gemrat              weights
      |                    |
      |________scores______|
                 |
                 |                   Boltzmann
      temp --> softmax --> probs --> pweights
```

The Markowitz weights may suggest that we bet against the
consistently poor performance of some agents.
We shall generally regard the weights as advisory,
taking what suits us and renormalizing.

To summarize the information set so far, we cast
the agents in a game, each with some score.
When the game consists of multiple rounds,
we can use tools from reinforcement learning
to help us make the best sequential decisions.

The softmax function is fed the scores to compute
the probability of a particular agent being the expert.
This function takes temperature as a diffusion parameter,
that is, an optimal way to diversify our bets across possible experts.
The theory here is due to Ludwig **Boltzmann** and his work on
statistical mechanics and entropy.
But the temperature setting can also be seen as a Bayesian
way to express the overall uncertainty involved with
estimating the various measures.

Finally, those probabilities are combined with our
renormalized weights to arrive at "pweights,"
our portfolio weights.

In [1]:
from fecon235.fecon235 import *

In [2]:
#  PREAMBLE-p6.15.1223d :: Settings and system details
from __future__ import absolute_import, print_function, division
system.specs()
pwd = system.getpwd()   # present working directory as variable.
print(" ::  $pwd:", pwd)
#  If a module is modified, automatically reload it:
%load_ext autoreload
%autoreload 2
#       Use 0 to disable this feature.

#  Notebook DISPLAY options:
#      Represent pandas DataFrames as text; not HTML representation:
import pandas as pd
pd.set_option( 'display.notebook_repr_html', False )
from IPython.display import HTML # useful for snippets
#  e.g. HTML('<iframe src=http://en.mobile.wikipedia.org/?useformat=mobile width=700 height=350></iframe>')
from IPython.display import Image 
#  e.g. Image(filename='holt-winters-equations.png', embed=True) # url= also works
from IPython.display import YouTubeVideo
#  e.g. YouTubeVideo('1j_HxD4iLn8', start='43', width=600, height=400)
from IPython.core import page
get_ipython().set_hook('show_in_pager', page.as_hook(page.display_page), 0)
#  Or equivalently in config file: "InteractiveShell.display_page = True", 
#  which will display results in secondary notebook pager frame in a cell.

#  Generate PLOTS inside notebook, "inline" generates static png:
%matplotlib inline   
#          "notebook" argument allows interactive zoom and resize.

 ::  Python 2.7.13
 ::  IPython 5.1.0
 ::  jupyter_core 4.2.1
 ::  notebook 4.1.0
 ::  matplotlib 1.5.1
 ::  numpy 1.11.0
 ::  scipy 0.17.0
 ::  sympy 1.0
 ::  pandas 0.19.2
 ::  pandas_datareader 0.2.1
 ::  Repository: fecon235 v5.17.0603 devPrtf
 ::  Timestamp: 2017-07-09T01:48:51Z
 ::  $pwd: /media/yaya/virt15h/virt/dbx/Dropbox/ipy/fecon235/nb


We will be stepping through the algorithm with real data,
so that the code later will not look like a black box.
The numerical output along the way can guide and test
the construction of a Boltzmann portfolio.

## Download data and construct a dataframe

We retrieve the following data of daily frequency
representing equities worldwide and gold by five ETF securities: 

In [3]:
#  Convenient dictionary set in fecon235.py,
#  where the keys are world regions,
#  and the values fecon235 stock slang:
world4d

#  Gold is included as a proxy "safe-haven"
#  but serves also to test the covariance structure.

{'America': 's4spy',
 'Emerging': 's4eem',
 'Europe': 's4ezu',
 'Gold': 's4gld',
 'Japan': 's4ewj'}

In [4]:
#  Or manually specify your own dictionary here:
prices_dic = world4d

In [5]:
#  Download data into a dataframe, alphabetically by key:
prices = groupget( prices_dic, maxi=3650 )
#  ... about ten years worth.

 ::  Retrieved from Google Finance: SPY
 ::  Retrieved from Google Finance: EEM
 ::  Retrieved from Google Finance: EZU
 ::  Retrieved from Google Finance: GLD
 ::  Retrieved from Google Finance: EWJ


### ========= Specify DATES for  [start:end] =========

We can analyze different epochs of history
by executing **"Cell" > "Run All Below"** from the Jupyter menu,
*without downloading the data again*.

**Include CONSTANTS for the remainder of this notebook.**

In [6]:
#  Set the start and end dates:
start = '2011-01-01'
end = '2017-06-26'

#  CONSTANTS
MIN_weight = 0.01
TEMPERATURE = 55

### Summary statistics: "prices" dataframe

In [7]:
stats( prices[start:end] )

           America     Emerging       Europe         Gold        Japan
count  1630.000000  1630.000000  1630.000000  1630.000000  1630.000000
mean    177.792006    40.000503    35.414362   132.703399    44.972448
std      35.941680     4.116889     4.192482    20.404781     5.006797
min     109.930000    28.250000    24.990000   100.500000    34.600000
25%     140.490000    37.542500    32.770000   117.190000    40.320000
50%     186.300000    40.295000    35.465000   125.515000    46.000000
75%     207.995000    42.667500    38.790000   152.687500    48.595000
max     244.660000    50.210000    44.190000   184.590000    54.900000

 ::  Index on min:
America    2011-10-03
Emerging   2016-01-20
Europe     2012-07-24
Gold       2015-12-17
Japan      2012-06-01
dtype: datetime64[ns]

 ::  Index on max:
America    2017-06-19
Emerging   2011-04-26
Europe     2014-06-06
Gold       2011-08-22
Japan      2017-06-02
dtype: datetime64[ns]

 ::  Head:
            America  Emerging  Europe    Gold

## Geometric mean rate

David E. Shaw, famous for his proprietary hedge fund, remarked that 
one of the most important equations in finance is the penalization 
of arithmetic mean by one-half of variance:

$$ g = \mu - \frac{\sigma^2}{2} $$

which turns out to be a second-order approximation of geometric mean rate.
It is good enough to maximize, before considering 
mean-variance trade-offs.

However, many assets have leptokurtotic returns ("fat-tails") and so
a more accurate approximation of the geometric mean rate is needed
which considers the fourth central moment called *kurtosis* as risk.
Details are given on Gaussian mixtures in our research
at https://git.io/gmix

For maximimizing wealth over many periods, the geometric mean rate
as an objective metric should be optimized, rather than the
arithmetic mean rate (which precludes risk).

A Boltzmann portfolio maximizes the weighted geometric mean rate
of its underlying assets. We shall assume that the covariance structure,
e.g. the correlations, of the geometric mean rates is similar
to the arithmetic mean rates.

The source code shown by `groupgemrat??` tells us the output format:
the **geometric** mean return, followed by 
the **arithmetic mean return, volatility, and Pearson kurtosis**, 
then yearly frequency, sample size, and key -- in list format.

In [8]:
#  Geometric mean rates, non-overlapping, annualized:
gems = groupgemrat( prices[start:end], yearly=256, order=False, n=4 )
gems

[[9.1626, 10.2097, 14.7321, 7.6681, 256, 1629, 'America'],
 [-4.9559, -2.1686, 21.8959, 6.1498, 256, 1629, 'Emerging'],
 [-1.4118, 2.0996, 24.0893, 9.2461, 256, 1629, 'Europe'],
 [-4.0893, -2.4126, 17.0602, 8.7778, 256, 1629, 'Gold'],
 [1.335, 3.1714, 18.5607, 6.7887, 256, 1629, 'Japan']]

The fourth element in each sublist gives us the kurtosis statistic
where 3 is theoretically expected if the distribution is Gaussian.
Anything much higher is considered "leptokurtoic."

## Portfolio characteristics

At this point, if we were constrained to pick a single agent,
we would pick the one with highest geometric mean rate.
If short sales were permitted, we would need to
evaluate the absolute values of the geometric mean rates.

This constrained single-period example makes it clear that
we are merely estimating the best allocation of a portfolio,
not the timing or size of the trade. Extreme compositions could
signal a *bubble* or *anti-bubble*, and a discretionary
warning should be issued by the code at the decision stage.

In [9]:
#  By construction, keys will be alphabetically sorted:
keys = list(prices.columns)
keys

['America', 'Emerging', 'Europe', 'Gold', 'Japan']

In [10]:
#  Gather just the geometric mean rates into an array:
rates = np.array([item[0] for item in gems]).reshape(len(gems), 1)
rates

array([[ 9.1626],
       [-4.9559],
       [-1.4118],
       [-4.0893],
       [ 1.335 ]])

In [11]:
#  Compute the COVARIANCE matrix V:
V = covdiflog( prices[start:end] )

In [12]:
#  Values within the covariance matrix itself is hard
#  to interpret, so we show the Pearson correlation coefficients,
#  rounded to n decimal places:
print( cov2cor(V, n=2) )

[[ 1.    0.82  0.82 -0.03  0.7 ]
 [ 0.82  1.    0.79  0.12  0.67]
 [ 0.82  0.79  1.    0.05  0.66]
 [-0.03  0.12  0.05  1.   -0.  ]
 [ 0.7   0.67  0.66 -0.    1.  ]]


This correlation matrix pertains to the differential
between logged prices, *not* the prices themselves
(which was summarized earlier).

We displayed the correlation matrix as a courtesy because
the covariance matrix itself does not have such a
readable interpretation of the rates of return.

Mathematically and computationally, we shall work henceforth
with $V$, the covariance matrix.

As expected, the equities group is inter-correlated,
whereas Gold rates stand apart.

## Weights from the covariance matrix

We now turn our attention to the weights associated with the **Global
Minimum Variance Portfolio**. Its derivation is well-known,
e.g. Cochrane (2005), chp. 5, p.83:

$$ \mathbf{w} = \frac{V^{-1}\mathbf{1}} { \mathbf{1}^\top V^{-1} \mathbf{1} } $$

Note that the weights are solely dependent on the covariance matrix $V$.
There are no constraints involved.

The Boltzmann portfolio is *informed* by the Lagrangian formulation
of the covariance structure, not in the static single-period sense,
but rather in its dynamic evolution over time.
Weights will be revised by an online algorithm which
tracks covariance, e.g. the Kalman filter, without
performing quadratic optimization offline.

We shall take only what we truly need in terms of weight coefficients
(to be handled by `trimit()` later).

In [13]:
##  Uncomment to see defined function written in numpy...
#  weighcov??

In [14]:
globalw = weighcov( V )
globalw

array([[ 0.87034542],
       [-0.2267291 ],
       [-0.19612603],
       [ 0.40540278],
       [ 0.14710693]])

The weights at this point are for the
*Global Minimum Variance Porfolio* (GMVP).
Negative weights imply shorting the underlying asset.

## Deciding on the expert(s)

We can now assign scores to our agents which will help to discern the expert(s).
A score will be the individual geometric mean rate weighted by the its GMVP weight.

In [15]:
#  Using arrays, we are multiplying element-wise:
scores_gmvp = globalw * rates
scores_gmvp

array([[ 7.97462698],
       [ 1.12364674],
       [ 0.27689073],
       [-1.65781357],
       [ 0.19638775]])

In [16]:
#  Expected GLOBAL portfolio return with UNRESTRICTED short sales:
np.sum(scores_gmvp)

7.9137386200179298

The weights indicating short positions are usually
paired with negative geometric returns.

Considering the covariance matrix,
an asset which has consistently very poor growth performance
may obtain the best score if shorting is permitted.

## Trimming weights: Dealing with short sales

Negative weights imply the underlying assets should be shorted.
A Boltzmann portfolio only considers the weights as ***advisory***
in recognition of the fact that the covariance structure is unstable.

We may want to limit short sales at -0.30 weight, or perhaps ignore tiny
positions for rebalancing purposes in a multiple-period setting.

We earlier specified a threshold weight: `MIN_weight`.
The `level` argument resets weights under the floor to a specific value,
here we pick zero as level.
The trimmed weights will then need to be renormalized.

In [17]:
trimit?

Signature: trimit(it, floor, level)
Docstring: For an iterable, accept values > floor, else set to level.
File:      ~/Dropbox/ipy/fecon235/lib/ys_prtf_boltzmann.py
Type:      function


In [18]:
weights = trimit( globalw, MIN_weight, 0 )
weights

array([[ 0.87034542],
       [ 0.        ],
       [ 0.        ],
       [ 0.40540278],
       [ 0.14710693]])

In [19]:
weights = renormalize(weights)
weights

array([[ 0.61168942],
       [ 0.        ],
       [ 0.        ],
       [ 0.28492203],
       [ 0.10338855]])

In [20]:
scores_rentrim = weights * rates
scores_rentrim

array([[ 5.60466546],
       [-0.        ],
       [-0.        ],
       [-1.16513167],
       [ 0.13802371]])

In [21]:
#  Expected portfolio return with rentrim weights:
np.sum(scores_rentrim)

4.5775574975845643

## Introducing Boltzmann and softmax

Suppose there are $N$ agents, and we want to estimate the probabilities
of an agent being the best in the game. So far our computations have not
directly estimated probabilities, instead real-valued scores were produced.
However, from these scores one can compute the probabilities using the
softmax function. We can work with functions which map to the unconstrained space
of scores, and then map those scores to the space of probability vectors.
One can view this as a generalization of logistic regression.

In statistical physics, the softmax function gives the probability of
an atom being found in a quantum state of energy when the atom is part of an
ensemble that has reached thermal equilibrium.
This is known as the **Boltzmann** distribution.

In the field of reinforcement learning, the softmax function is used to
convert values into action probabilities.
A positive parameter $\tau$ called the temperature is introduced
to divide through each value. It is a scaling operation such that
high temperatures cause corresponding actions to be equi-probable.
Low temperatures cause a greater difference in selection probability
for actions that differ in their value estimates.
At low temperatures, the probability of the action with the
highest expected reward tends to 1.

Suppose $\mathbf{p}$ is our softmax vector, then let its j-th element be:

$$ p_j = \frac {\exp({s_{j}/\tau})} {\sum_{i=1}^{N}\exp({s_{i}/\tau})} $$

where $s_{j}$ is the score for the j-th asset.
[Note: our softmax() code is numerically stabilized for very large scores,
and uses a scaled version of $\tau$ called `temp`.]
See References below for more details.
Mathematical properties and justification for portfolios
will be given in Part 4 of our notebook series.

Aside from its theoretical pedigree, **why would softmax be helpful
in optimizing the terminal value of our portfolio?**
Its elements, interpreted as probabilities, are essentially a function
of the *exponential growth* associated with the temperature
dampened scores. In other words, the *probabilities will tilt
favorably towards assets with the greatest growth potential.*

Softmax is also known as the "***normalized exponential function***."

In [22]:
problist = softmax( scores_rentrim, temp=TEMPERATURE )[-1]
probs = np.array( problist ).reshape(len(problist), 1)
probs

array([[ 0.7041],
       [ 0.0801],
       [ 0.0801],
       [ 0.051 ],
       [ 0.0846]])

It is important to note that probs here used a constant `TEMPERATURE` value.
In practice, the temperature can be varied over multiple time periods
(to be discussed further in Part 2).

*Knowing which agents are likely to be experts, we can now
feedback this information to recompute the weights.*

In [23]:
pweights = probs * weights
pweights

array([[ 0.43069052],
       [ 0.        ],
       [ 0.        ],
       [ 0.01453102],
       [ 0.00874667]])

Renormalization of pweights is necessary, but also trimit() is helpful,
followed by renormalize() again -- to satisfy our own trading restrictions.

In [24]:
pweights = renormalize(trimit(renormalize(pweights), MIN_weight, 0))
pweights

array([[ 0.94872395],
       [ 0.        ],
       [ 0.        ],
       [ 0.0320089 ],
       [ 0.01926714]])

In [25]:
scores = pweights * rates
scores

array([[ 8.6927781 ],
       [-0.        ],
       [-0.        ],
       [-0.130894  ],
       [ 0.02572164]])

In [26]:
#  Expected geometric mean rate for our Boltzmann portfolio:
np.sum(scores)

8.5876057401765546

## SUMMARY

Hopefully, our flowchart in the Introduction has become explicitly clear:


```
    prices ---> cov ---> globalw
      |                    |
      |                  trimit  <-- floor
      |                  renormalize
      |                    |
      v                    v
      |                    |
    gemrat              weights
      |                    |
      |________scores______|
                 |
                 |                   Boltzmann
      temp --> softmax --> probs --> pweights



```

The algorithmic summary is given by `boltzportfolio()`, and the
supporting functions are given in the module `lib/ys_prtf_boltzmann.py`,
The condensed versions will be thoroughly employed in Part 2
of the notebook series.

In [27]:
boltzportfolio??

Signature: boltzportfolio(dataframe, yearly=256, temp=55, floor=0.01, level=0, n=4)
Source:   
def boltzportfolio(dataframe, yearly=256, temp=55, floor=0.01, level=0, n=4):
    '''MAIN: SUMMARY of Boltzmann portfolio, rounded to n-decimal places.
       Return list where computed values are Python floats, not array type, e.g.
           [2.7833,
            [[0.6423, 2.05, 'America'],
             [0.0, -11.17, 'Emerging'],
             [0.0, -10.47, 'Europe'],
             [0.3577, 4.1, 'Gold'],
             [0.0, -4.99, 'Japan']]]
       The portfolio's geometric mean rate is included first.
       Each sub-sublist will consist of weight, rate, and key.
       The order of keys from the dataframe is preserved.
    '''
    rates = gemratarr(dataframe, yearly)
    globalw = weighcovdata(dataframe)
    weights = rentrim(globalw, floor, level)
    pweights = weighsoft(weights, rates, temp, floor, level)
    #      ---- so far should be the same as boltzweigh()
    scores = pweights * rat

***The numerical computations of this entire notebook,
including the inversion of the covariance matrix
and the softmax evaluation, can be replicated as follows:***

In [28]:
#  See the docstring to decipher the format:
boltzportfolio(prices[start:end], yearly=256, temp=TEMPERATURE, floor=MIN_weight, level=0)

[8.5876,
 [[0.9487, 9.1626, 'America'],
  [0.0, -4.9559, 'Emerging'],
  [0.0, -1.4118, 'Europe'],
  [0.032, -4.0893, 'Gold'],
  [0.0193, 1.335, 'Japan']]]

Noteworthy: Gold which had a *negative* geometric mean rate for itself
receives a *positive* pweight greater than Japan,
due to the covariance structure and our trimit specification. 

---

## References

- John H. Cochrane, 2005 revised ed., *Asset Pricing*, Princeton U. Press.

- On softmax:
    - https://en.wikipedia.org/wiki/Softmax_function
    - https://compute.quora.com/What-is-softmax
    - http://eli.thegreenplace.net/2016/the-softmax-function-and-its-derivative
    - http://cs231n.github.io/linear-classify/#softmax
    - https://en.wikipedia.org/wiki/Reinforcement_learning